# 데이터 전처리 이해와 실무

## `데이터 정제: 이상치 데이터 처리`

### 이상치 다루기(강의 교안)
1. 이상치 확인
   - Z-SCORE, IQR(Interquartile Range)
2. 이상치 처리
   - 삭제, 대체

### 실습 데이터
- 이상치 강좌 실습을 위한 생성 데이터: 기사별 클릭 수 데이터

## 0. 활용 패키지 및 데이터 로딩

In [1]:
import numpy as np
import pandas as pd

In [21]:
import pandas as pd
import numpy as np
import random
import string
from scipy.stats import zscore

# 생성할 데이터의 행 수
num_rows = 17734

# 1. 카테고리와 저널 데이터 정의
categories = ['사설', '사회', '공학', '증권', '부동산', '경제', '사설', '정치', '부동산', '사회', '스포츠', '경제', '사설', '사회', '연예']
journals = ['C일보', 'B일보', 'C일보', 'E뉴스', 'B일보', 'B일보', 'C일보', 'D일보', 'D일보', 'B일보', 'B일보', 'B일보', 'B일보', 'B일보', 'B일보']

# 2. 무작위 ID 생성 함수
def generate_random_id(length=10):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# 3. 로그 정규 분포 기반 클릭 수 생성
def generate_clicks_log_normal(num_rows):
    np.random.seed(0)  # 재현 가능한 결과를 위해 시드 설정
    mean = np.log(1000)
    sigma = 1.0
    clicks = np.random.lognormal(mean, sigma, num_rows)
    clicks = np.clip(clicks, 9, 433992)  # 최소값과 최대값 설정
    return clicks.astype(int)

# 4. IQR 조정 함수
def adjust_iqr(clicks, desired_iqr):
    current_iqr = np.percentile(clicks, 75) - np.percentile(clicks, 25)
    scale_factor = desired_iqr / current_iqr
    clicks = (clicks - np.median(clicks)) * scale_factor + np.median(clicks)
    return np.clip(clicks, 9, 433992).astype(int)

# 데이터프레임 생성
initial_clicks = generate_clicks_log_normal(num_rows)
adjusted_clicks = adjust_iqr(initial_clicks, desired_iqr=804)

# z-score 조정
adjusted_zscores = zscore(adjusted_clicks)

# 데이터프레임 생성
data = {
    'category': [random.choice(categories) for _ in range(num_rows)],
    'Journal': [random.choice(journals) for _ in range(num_rows)],
    'article_id': [generate_random_id() for _ in range(num_rows)],
    'num_click': adjusted_clicks
}

df = pd.DataFrame(data)

# 데이터프레임을 CSV 파일로 저장
df.to_csv('./data/article_click.csv', index=False)

# 확인용 통계 출력
print("IQR:", np.percentile(adjusted_clicks, 75) - np.percentile(adjusted_clicks, 25))
print("Mean:", df['num_click'].mean())
print("Std Dev:", df['num_click'].std())
print("Z-Score Min:", adjusted_zscores.min())
print("Z-Score Max:", adjusted_zscores.max())

IQR: 804.0
Mean: 1348.0526108041051
Std Dev: 1185.4509119071472
Z-Score Min: -0.7634884922131655
Z-Score Max: 20.510029275472696


In [3]:
# 데이터 로딩 및 개요 확인
click_data = pd.read_csv("./data/article_click.csv") # 다른 데이터 불러올 때는 , encoding = 'cp949' 넣어줘야 함!
click_data.head(15)

,category,Journal,article_id,num_click
0,사설,E뉴스,9NGvJHSxMt,3719
1,스포츠,D일보,qujSGSSVQj,1271
2,사회,C일보,sOx7VDWpMN,1929
3,사설,B일보,TOh0A0Dj0Q,5728
4,사회,C일보,hHxD2MJ6m9,4077
5,사회,B일보,n1EZb6tJcJ,642
6,연예,B일보,zHTsEDWUxZ,1887
7,사설,B일보,lFGGmpHhfa,914
8,공학,D일보,HBMJVNlfGC,938
9,사설,D일보,gCUjTevkHF,1279


In [4]:
# 데이터 개요 파악
click_data.info() # 클릭수만 연속형이고 나머지는 다 범주형 자료

# data copy
click_copy = click_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17734 entries, 0 to 17733
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    17734 non-null  object
 1   Journal     17734 non-null  object
 2   article_id  17734 non-null  object
 3   num_click   17734 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 554.3+ KB


In [5]:
# describe 함수 활용 기반의 수치형 변수인 num_click(클릭 횟수) 컬럼 요약 통계 확인
click_copy['num_click'].describe()

count    17734.000000
mean      1348.052611
std       1185.450912
min        443.000000
25%        717.000000
50%        986.000000
75%       1521.000000
max      25661.000000
Name: num_click, dtype: float64

- 최대 클릭 수는 433992회, 최소 9회 클릭되었음
- 75% 수준의 클릭 수는 1000회 정도의 결과를 보임에 따라 일부 이상치 데이터 존재하는 것으로 보여짐

## 1. 이상치 처리하기

### 1-1. 이상치 확인 방안 및 삭제하기: `Z-Score`
1. Z-Score : Z는 (해당관측치 - 관측치 변수 평균) / (관측치 변수의 표준편차)
2. IQR

In [6]:
# Z-score 컬럼 생성
click_copy['z_score'] = (click_copy['num_click'] - np.mean(click_copy['num_click'])) / np.std(click_copy['num_click'])
click_copy.head()

,category,Journal,article_id,num_click,z_score
0,사설,E뉴스,9NGvJHSxMt,3719,2.000095
1,스포츠,D일보,qujSGSSVQj,1271,-0.065000
2,사회,C일보,sOx7VDWpMN,1929,0.490078
3,사설,B일보,TOh0A0Dj0Q,5728,3.694856
4,사회,C일보,hHxD2MJ6m9,4077,2.302098


In [7]:
# 원본 데이터 내 z-score 확인
click_copy.describe() # min : -0.6936 # max : 2.23 --> 만약 min : -0.228, max : 92.239면 양의 방향으로 이상치 존재

,num_click,z_score
count,17734.000000,1.773400e+04
mean,1348.052611,-1.943235e-17
std,1185.450912,1.000028e+00
min,443.000000,-7.634885e-01
25%,717.000000,-5.323463e-01
50%,986.000000,-3.054220e-01
75%,1521.000000,1.458958e-01
max,25661.000000,2.051003e+01


- -3과 3 사이를 벗어난 값을 삭제해보겠다.

In [8]:
# 이상치 처리하기(삭제)
# z-score 기반 이상치 제거 후 데이터 차원 확인
click_copy = click_copy[(click_copy['z_score'] < 3) & (click_copy['z_score'] > -3)]

# 데이터 개요 확인
click_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17398 entries, 0 to 17733
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   category    17398 non-null  object 
 1   Journal     17398 non-null  object 
 2   article_id  17398 non-null  object 
 3   num_click   17398 non-null  int64  
 4   z_score     17398 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 815.5+ KB


In [9]:
# 이상치 제거 후 데이터 요약 통계 확인
# z-score의 Min, Max 가 절댓값 3을 초과하지 않음을 확인
click_copy.describe()

,num_click,z_score
count,17398.000000,17398.000000
mean,1233.316358,-0.096790
std,767.232935,0.647226
min,443.000000,-0.763488
25%,713.000000,-0.535721
50%,973.000000,-0.316389
75%,1474.000000,0.106247
max,4899.000000,2.995525


In [10]:
# 한편, scipy(싸이파이) 패키지 내에서 z-score 함수를 별도로 제공하고 있다. 
# 별도 수식하지 않아도 편하게 구할 수 있는 방법!
from scipy.stats import zscore
click_copy = click_data.copy()

click_copy['z_score'] = zscore(click_copy['num_click'])
click_copy = click_copy[(click_copy['z_score'] < 3) & (click_copy['z_score'] > -3)]
print(click_copy.shape)

(17398, 5)


In [11]:
# 기존 결과와 비교
click_copy.describe() # 동일함을 확인 가능!

,num_click,z_score
count,17398.000000,17398.000000
mean,1233.316358,-0.096790
std,767.232935,0.647226
min,443.000000,-0.763488
25%,713.000000,-0.535721
50%,973.000000,-0.316389
75%,1474.000000,0.106247
max,4899.000000,2.995525


### 1-2. 이상치 확인 방안 및 삭제하기: `IQR`
1. Z-Score : Z는 (해당관측치 - 관측치 변수 평균) / (관측치 변수의 표준편차)
2. IQR

In [12]:
# IQR 판단 기반 이상치 처리
# 원 데이터 COPY
click_copy = click_data.copy()

In [13]:
# 1, 3 분위수 (Q1, Q3) 구하기
q1 = click_copy['num_click'].quantile(0.25)
q3 = click_copy['num_click'].quantile(0.75)

# IQR 구하기 (Interquartile Range)
iqr = (q3 - q1)
iqr

804.0

In [14]:
# IQR 기반 이상치 제거하기
click_copy = click_copy[(click_copy['num_click'] > (q1 - 1.5*iqr)) & (click_copy['num_click'] < (q3 + 1.5*iqr))]
click_copy.describe()

,num_click
count,16358.000000
mean,1089.505624
std,509.026659
min,443.000000
25%,699.000000
50%,935.000000
75%,1343.000000
max,2725.000000


In [15]:
# 이상치 제거 후 데이터 확인
print(np.shape(click_copy))

(16358, 4)


### 2. 이상치 대체하기
- 만일 특정 목적에 따라서 기사별 클릭 횟수의 최대값을 사전에 정의하고, 그 이상의 클릭 횟수를 지닌 경우에는 기사를 통한 수익의 의미가 크게 의미가 없다고 판단되어서 데이터를 변경해야 한다면, 데이터 삭제 없이 데이터 이상치를 일괄적으로 처리 가

In [17]:
# 이상치 대체
# 원데이터 copy
click_copy = click_data.copy()

In [18]:
# 대체할 기준 정의
max_click = 1000

In [19]:
# 정의된 기준으로 대체 후 비교를 위해 컬럼 복사
click_copy['new_num_click'] = click_copy['num_click']

# 이상치 대체
click_copy.loc[click_copy['new_num_click'] > max_click, 'new_num_click'] = max_click
click_copy.describe()

,num_click,new_num_click
count,17734.000000,17734.000000
mean,1348.052611,862.033100
std,1185.450912,169.283572
min,443.000000,443.000000
25%,717.000000,717.000000
50%,986.000000,986.000000
75%,1521.000000,1000.000000
max,25661.000000,1000.000000


In [20]:
click_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17734 entries, 0 to 17733
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category       17734 non-null  object
 1   Journal        17734 non-null  object
 2   article_id     17734 non-null  object
 3   num_click      17734 non-null  int64 
 4   new_num_click  17734 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 692.9+ KB


- 일반적으로 이상치 대체 및 변경은 기존 도메인 지식 및 현업 담당자와의 협의를 통해 진행
- 위 데이터 예제에서 기사 클릭 수의 Max 값을 1000회로 정의하고 1000회 이상의 수 값은 모두 1000회로 대체